# Bayesian Logistic Regression in Edward

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from io import BytesIO
from timeit import default_timer as timer

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pickle

import edward as ed
from edward.models import Bernoulli, Normal, Empirical

from utils import compute_metrics, SEED

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
ed.set_seed(42)

## 1. Data

Credit Card Fraud Detection [1]

In [2]:
%gcs read --object "gs://thesis-203306/data/creditcard.csv" --variable csv_as_bytes

In [3]:
df = pd.read_csv(BytesIO(csv_as_bytes))

In [4]:
y = np.array(df.Class.tolist())     
df = df.drop('Class', 1)

df = df.drop('Time', 1)     
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
X = np.array(df.values)   

Toy Dataset 

Adapted from [2]

In [3]:
def build_toy_dataset(N, D=1, noise_std=0.1):    
    X = np.concatenate([np.linspace(-6, -5, num=5), np.linspace(2, 6, num=N-5)])
    y = np.tanh(X) + np.random.normal(0, noise_std, size=N)
    y[y < 0.5] = 0
    y[y >= 0.5] = 1
    X = (X - 4.0) / 4.0
    X = X.reshape((N, D))
    return X, y.astype(int)
  
N1 = 100
D1 = 1
X1, y1 = build_toy_dataset(N1, D=1, noise_std=0.1)

## 2. Inference

### Metropolis-Hastings

In [7]:
def _ed_MH(X, y, iters=2000, burn=1000):
  """
  Runs Edward's Metropolis-Hastings algorithm on one dataset
  """
  # The model
  n_samples, n_dim = X.shape
  X_e = tf.placeholder(tf.float32, [n_samples, n_dim])
  w_e = Normal(loc=tf.zeros(n_dim), scale=3.0 * tf.ones(n_dim))
  b_e = Normal(loc=0.0, scale=3.0)
  y_e = Bernoulli(logits=ed.dot(X_e, w_e) + b_e)
    
  qw = Empirical(params=tf.Variable(tf.zeros([iters, n_dim])))
  qb = Empirical(params=tf.Variable(tf.zeros([iters])))
  
  w_proposal = Normal(loc=w_e, scale=0.1)
  b_proposal = Normal(loc=b_e, scale=0.1)
 
  inference = ed.MetropolisHastings({w_e: qw, b_e: qb}, {w_e: w_proposal, b_e: b_proposal}, data={X_e: X, y_e: y})
    
  start = timer()
  inference.run()
  time = timer() - start
    
  return qw.params.eval()[:burn], qb.params.eval()[:burn], time

In [8]:
def ed_MH(X, y, filename, iters=2000, burn=1000, seeds=SEED):
  """
  Runs Edward's Metropolis-Hastings algorithm
  """
  for seed in seeds:
    print(seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    qw, qb, time = _ed_MH(X_train, y_train, iters, burn)
    
    accuracy = []
    precision = []
    recall = []
    F1 = []
    
    for ww, bb in zip(qw, qb):
      a, p, r, f = compute_metrics(ww, bb, X_test, y_test)
      accuracy.append(a)
      precision.append(p)
      recall.append(r)
      F1.append(f)
    
    results = {'time': time, 'w': qw, 'b': qb, 'iters': iters, 'burn': burn, 
              'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': F1}
    with open('results/edward/{}_{}.pkl'.format(filename, seed), 'wb') as f:
      pickle.dump(results, f)
  print('Done')

In [ ]:
# Credit card fraud - short chain
ed_MH(X, y, 'mh_credit', iters=2000, burn=1000, seeds=SEED)

In [ ]:
# Credit card fraud - long chain
ed_MH(X, y, 'mh_credit_long', iters=8000, burn=4000, seeds=SEED)

### KLqp

In [23]:
def _ed_KLqp(X,y,N,D,iters):
  """
  Runs Edward's Klqp algorithm on one dataset
  """
  X_e = tf.placeholder(tf.float32, [N, D])
  w_e = Normal(loc=tf.zeros(D), scale=3.0 * tf.ones(D))
  b_e = Normal(loc=0.0, scale=3.0)
  y_e = Bernoulli(logits=ed.dot(X_e, w_e) + b_e)
    
  with tf.variable_scope('qw_loc', reuse=tf.AUTO_REUSE):
    qw_loc = tf.get_variable("qw_loc", [D])
  with tf.variable_scope('qw_scale', reuse=tf.AUTO_REUSE):
    qw_scale = tf.nn.softplus(tf.get_variable("qw_scale", [D]))
  with tf.variable_scope('qb_loc', reuse=tf.AUTO_REUSE):
    qb_loc = tf.get_variable("qb_loc", []) + 10.0
  with tf.variable_scope('qb_scale', reuse=tf.AUTO_REUSE):
    qb_scale = tf.nn.softplus(tf.get_variable("qb_scale", []))

  qw = Normal(loc=qw_loc, scale=qw_scale)
  qb = Normal(loc=qb_loc, scale=qb_scale)

  inference = ed.KLqp({w_e: qw, b_e: qb}, data={X_e: X, y_e: y})
  start = timer()
  inference.run(n_iter=iters)
  end = timer()
  
  return qw, qb, end-start

In [24]:
def ed_KLqp(X, y, filename, seeds=SEED):
  """
  Runs Edward's KLqp algorithm for each seed
  """
  for seed in seeds:
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=seed)
    N,D = X_train.shape
    
    iters = np.linspace(500, 10000, 5).astype(int)
    
    accuracy = []
    precision = []
    recall = []
    F1 = []
    times = []
    
    for it in iters:
      qw, qb, time = _ed_KLqp(X_train,y_train,N,D,it)
      w = qw.eval()
      b = qb.eval()
      a, p, r, f = compute_metrics(w, b, X_test,y_test)
      accuracy.append(a)
      precision.append(p)
      recall.append(r)
      F1.append(f)
      times.append(time)
    
    results = {'iters': iters, 'times': times, 'accuracy': accuracy, 'precision': precision,
               'recall': recall, 'F1': F1}
    with open('results/edward/{}_{}.pkl'.format(filename, seed), 'wb') as f:
      pickle.dump(results, f)
  print('Done')

In [ ]:
# Credit card fraud detection
ed_KLqp(X, y, 'vi_credit', seeds=SEED)

## References

[1] Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

[2] Edward [tutorial](http://edwardlib.org/tutorials/supervised-regression)

[3] Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.